# Apache Sqoop Lect. 1

## Transferring an Entire Table
### Problem

You have a table in a relational database (e.g., MySQL) and you need to transfer the table’s contents into Hadoop’s Distributed File System (HDFS).
### Solution

Importing one table with Sqoop is very simple: you issue the Sqoop import command and specify the database credentials and the name of the table to transfer.

## Specifying a Target Directory

### Problem

The previous example worked well, so you plan to incorporate Sqoop into your Hadoop workflows. In order to do so, you want to specify the directory into which the data should be imported.

### Solution

Sqoop offers two parameters for specifying custom output directories: --target-dir and --warehouse-dir. Use the --target-dir parameter to specify the directory on HDFS where Sqoop should import your data. F

### Discussion
If you want to run multiple Sqoop jobs for multiple tables, you will need to change the --target-dir parameter with every invocation. As an alternative, Sqoop offers another parameter by which to select the output directory. Instead of directly specifying the final directory, the parameter --warehouse-dir allows you to specify only the parent directory. Rather than writing data into the warehouse directory, Sqoop will create a directory with the same name as the table inside the warehouse directory and import data there. This is similar to the default case where Sqoop imports data to your home directory on HDFS, with the notable exception that the --warehouse-dir parameter allows you to use a directory other than the home directory. Note that this parameter does not need to change with every table import unless you are importing tables with the same name.
Caution

#### NOTE:
Just as with the --target-dir parameter, Sqoop will reject importing data when the final output directory already exists. In this case, the name is comprised of the directory name specified in --warehouse-dir and the name of a transferred table.

## Importing Only a Subset of Data
### Problem

Instead of importing an entire table, you need to transfer only a subset of the rows based on various conditions that you can express in the form of a SQL statement with a WHERE clause.
### Solution

Use the command-line parameter --where to specify a SQL condition that the imported data should meet. 

### Discussion

Sqoop will propagate the content of the --where parameter as is to all generated queries that fetch data. This provides a powerful ability by which to express any condition that your particular database server can process. Any special functions, conversions, or even user-defined functions can be used. Because the SQL fragment will be propagated into generated queries without any Sqoop processing, any invalid fragments may result in nonintuitive exceptions that are hard to debug. This parameter can be confusing for new Sqoop users.

When using the --where parameter, keep in mind the parallel nature of Sqoop transfers. Data will be transferred in several concurrent tasks. Any expensive function call will put a significant performance burden on your database server. Advanced functions could lock certain tables, preventing Sqoop from transferring data in parallel. This will adversely affect transfer performance. For efficient advanced filtering, run the filtering query on your database prior to import, save its output to a temporary table and run Sqoop to import the temporary table into Hadoop without the --where parameter.

## Using a File Format Other Than CSV
### Problem

The tab-separated CSV file that Sqoop uses by default does not suit your use case. You prefer a binary format over plain text.
### Solution

Sqoop supports three different file formats; one of these is text, and the other two are binary. The binary formats are Avro and Hadoop’s SequenceFile. You can enable import into SequenceFile using the --as-sequencefile parameter:

### Discussion

Binary formats have a few benefits over that of text files. First, binary formats are a natural fit for storing binary values like images or PDF documents. They are also more suited for storing text data if the data itself contains characters that are otherwise used as separators in the text file. Along with these benefits, there is one downside: in order to access the binary data, you need to implement extra functionality or load special libraries in your application.

The SequenceFile is a special Hadoop file format that is used for storing objects and implements the Writable interface. This format was customized for MapReduce, and thus it expects that each record will consist of two parts: key and value. Sqoop does not have the concept of key-value pairs and thus uses an empty object called NullWritable in place of the value. For the key, Sqoop uses the generated class. For convenience, this generated class is copied to the directory where Sqoop is executed. You will need to integrate this generated class to your application if you need to read a Sqoop-generated SequenceFile.

Apache Avro is a generic data serialization system. Specifying the --asavrodatafile parameter instructs Sqoop to use its compact and fast binary encoding format. Avro is a very generic system that can store any arbitrary data structures. It uses a concept called schema to describe what data structures are stored within the file. The schema is usually encoded as a JSON string so that it’s decipherable by the human eye. Sqoop will generate the schema automatically based on the metadata information retrieved from the database server and will retain the schema in each generated file. Your application will need to depend on Avro libraries in order to open and process data stored as Avro. You don’t need to import any special class, such as in the SequenceFile case, as all required metadata is embedded in the imported files themselves.

## Compressing Imported Data
### Problem

You want to decrease the overall size occupied on HDFS by using compression for generated files.
### Solution

Use the parameter --compress to enable compression:

### Discussion

Sqoop takes advantage of the inherent parallelism of Hadoop by leveraging Hadoop’s execution engine, MapReduce, to perform data transfers. As MapReduce already has excellent support for compression, Sqoop simply reuses its powerful abilities to provide compression options. By default, when using the --compress parameter, output files will be compressed using the GZip codec, and all files will end up with a .gz extension. You can choose any other codec using the --compression-codec parameter. The following example uses the BZip2 codec instead of GZip (files on HDFS will end up having the .bz2 extension):

Another benefit of leveraging MapReduce’s compression abilities is that Sqoop can make use of all Hadoop compression codecs out of the box. You don’t need to enable compression codes within Sqoop itself. That said, Sqoop can’t use any compression algorithm not known to Hadoop. Prior to using it with Sqoop, make sure your desired codec is properly installed and configured across all nodes in your cluster.
Caution

As Sqoop delegates compression to the MapReduce engine, you need to make sure the compressed map output is allowed in your Hadoop configuration. For example, if in the mapred-site.xml file, the property mapred.output.compress is set to false with the final flag, then Sqoop won’t be able to compress the output files even when you call it with the --compress parameter.

The selected compression codec might have a significant impact on subsequent processing. Some codecs do not support seeking to the middle of the compressed file without reading all previous content, effectively preventing Hadoop from processing the input files in a parallel manner. You should use a splittable codec for data that you’re planning to use in subsequent processing. Table 2-2 contains a list of splittable and nonsplittable compression codecs that will help you choose the proper codec for your use case.

## Speeding Up Transfers
### Problem

Sqoop is a great tool, and it’s processing bulk transfers very well. Can Sqoop run faster?
### Solution

For some databases you can take advantage of the direct mode by using the --direct parameter:

### Discussion

Rather than using the JDBC interface for transferring data, the direct mode delegates the job of transferring data to the native utilities provided by the database vendor. In the case of MySQL, the mysqldump and mysqlimport will be used for retrieving data from the database server or moving data back. In the case of PostgreSQL, Sqoop will take advantage of the pg_dump utility to import data. Using native utilities will greatly improve performance, as they are optimized to provide the best possible transfer speed while putting less burden on the database server. There are several limitations that come with this faster import. For one, not all databases have available native utilities. This mode is not available for every supported database. Out of the box, Sqoop has direct support only for MySQL and PostgreSQL.

Because all data transfer operations are performed inside generated MapReduce jobs and because the data transfer is being deferred to native utilities in direct mode, you will need to make sure that those native utilities are available on all of your Hadoop TaskTracker nodes. For example, in the case of MySQL, each node hosting a TaskTracker service needs to have both mysqldump and mysqlimport utilities installed.

Another limitation of the direct mode is that not all parameters are supported. As the native utilities usually produce text output, binary formats like SequenceFile or Avro won’t work. Also, parameters that customize the escape characters, type mapping, column and row delimiters, or the NULL substitution string might not be supported in all cases.
See Also

Sqoop also supports the pg_bulkload utility for PostgreSQL via a special build-in connector. You can find more information about that in Exporting into PostgreSQL Using pg_bulkload.

## Overriding Type Mapping
### Problem

The default type mapping that Sqoop provides between relational databases and Hadoop usually works well. You have use cases requiring you to override the mapping.
### Solution

Use Sqoop’s ability to override default type mapping using the parameter --map-column-java. For example, to override the type of column id to Java type Long:

### Discussion

The parameter --map-column-java accepts a comma separated list where each item is a key-value pair separated by an equal sign. The exact column name is used as the key, and the target Java type is specified as the value. For example, if you need to change mapping in three columns c1, c2, and c3 to Float, String, and String, respectively, then your Sqoop command line would contain the following fragment:

sqoop import --map-column-java c1=Float,c2=String,c3=String ...

An example of where this parameter is handy is when your MySQL table has a primary key column that is defined as unsigned int with values that are bigger than 2 147 483 647. In this particular scenario, MySQL reports that the column has type integer, even though the real type is unsigned integer. The maximum value for an unsigned integer column in MySQL is 4 294 967 295. Because the reported type is integer, Sqoop will use Java’s Integer object, which is not able to contain values larger than 2 147 483 647. In this case, you have to manually provide hints to do more appropriate type mapping.

Use of this parameter is not limited to overcoming MySQL’s unsigned types problem. It is further applicable to many use cases where Sqoop’s default type mapping is not a good fit for your environment. Sqoop fetches all metadata from database structures without touching the stored data, so any extra knowledge about the data itself must be provided separately if you want to take advantage of it. For example, if you’re using BLOB or BINARY columns for storing textual data to avoid any encoding issues, you can use the --column-map-java parameter to override the default mapping and import your data as String.

## Controlling Parallelism
### Problem

Sqoop by default uses four concurrent map tasks to transfer data to Hadoop. Transferring bigger tables with more concurrent tasks should decrease the time required to transfer all data. You want the flexibility to change the number of map tasks used on a per-job basis.
### Solution

Use the parameter --num-mappers if you want Sqoop to use a different number of mappers. For example, to suggest 10 concurrent tasks, you would use the following Sqoop command:

### Discussion

The parameter --num-mappers serves as a hint. In most cases, you will get the specified number of mappers, but it’s not guaranteed. If your data set is very small, Sqoop might resort to using a smaller number of mappers. For example, if you’re transferring only 4 rows yet set --num-mappers to 10 mappers, only 4 mappers will be used, as the other 6 mappers would not have any data to transfer.

Controlling the amount of parallelism that Sqoop will use to transfer data is the main way to control the load on your database. Using more mappers will lead to a higher number of concurrent data transfer tasks, which can result in faster job completion. However, it will also increase the load on the database as Sqoop will execute more concurrent queries. Doing so might affect other queries running on your server, adversely affecting your production environment. Increasing the number of mappers won’t always lead to faster job completion. While increasing the number of mappers, there is a point at which you will fully saturate your database. Increasing the number of mappers beyond this point won’t lead to faster job completion; in fact, it will have the opposite effect as your database server spends more time doing context switching rather than serving data.

The optimal number of mappers depends on many variables: you need to take into account your database type, the hardware that is used for your database server, and the impact to other requests that your database needs to serve. There is no optimal number of mappers that works for all scenarios. Instead, you’re encouraged to experiment to find the optimal degree of parallelism for your environment and use case. It’s a good idea to start with a small number of mappers, slowly ramping up, rather than to start with a large number of mappers, working your way down.
## Encoding NULL Values
### Problem

Sqoop encodes database NULL values using the null string constant. Your downstream processing (Hive queries, custom MapReduce job, or Pig script) uses a different constant for encoding missing values. You would like to override the default one.
### Solution

You can override the NULL substitution string with the --null-string and --null-non-string parameters to any arbitrary value. For example, use the following command to override it to \N:

### Discussion

Almost all relational databases allow you to define columns as optional, allowing the columns to be associated with no value. This missing information is usually referred to as the NULL value. For example, a BOOL column generally contains only two distinct values: true and false. Additionally, the column can contain the value NULL if during table creation you’ve explicitly allowed that column to be optional. Databases usually store the NULL value as an extra bit in addition to the column’s usual data. With Sqoop supporting the import of data to formats that don’t natively support the NULL value (e.g., CSV file), there is a need to encode the missing value into the data itself. By default, Sqoop uses the string constant null (lowercased) for representing the missing value. This default constant might not suit your needs if your data can contain the same string constant as a regular value or if your downstream processing is expecting a different substitution string constant.

To allow easier integration with additional Hadoop ecosystem components, Sqoop distinguishes between two different cases when dealing with missing values. For text-based columns that are defined with type VARCHAR, CHAR, NCHAR, TEXT, and a few others, you can override the default substitution string using the parameter --null-string. For all other column types, you can override the substitution string with the --null-non-string parameter. Some of the connectors might not support different substitution strings for different column types and thus might require you to specify the same value in both parameters.

Internally, the values specified in the --null(-non)-string parameters are encoded as a string constant in the generated Java code. You can take advantage of this by specifying any arbitrary string using octal representation without worrying about proper encoding. An unfortunate side effect requires you to properly escape the string on the command line so that it can be used as a valid Java string constant.

## Importing All Your Tables
### Problem

You would like to import all tables from your database at once using one command rather than importing the tables one by one.
### Solution

Rather than using the import tool for one table, you can use the import-all-tables tool. For example, to import all tables from our example database, you would use the following Sqoop command:

### Discussion

When using the import-all-tables tool, Sqoop will use catalog queries to retrieve a list of all tables in your database. Subsequently for each table, Sqoop will internally call the import tool to import your data to Hadoop. Tables will be imported in sequential order to avoid any unnecessary burden on the database server that would be created by importing several tables at the same time. With this one command, Sqoop will import the entire content of a given database to Hadoop.

If you need to import all but a few tables, you can use the parameter --exclude-tables that accepts a comma-separated list of table names that should be excluded from the bulk import. For example, if you need to import all tables from the database except cities and countries, you would use the following command:

Importing all tables will internally call the usual import tool for transferring each table. You’ll find that many of the import parameters can’t be used in conjunction with the import-all-tables tool.

For example, you can’t use the parameter --target-dir, as that would instruct Sqoop to import all tables into the same directory, resulting in a total file mess on HDFS.

Using the --warehouse-dir parameter is fine, as this parameter can be easily used for all imported tables. You can take advantage of the parameter --exclude-tables to skip importing tables that need special parameters; you can then import them separately using the import tool, which allows you to specify additional parameters.